# Chapter 26: Monad transformers

In [1]:
:!stack install hs-functors

In [2]:
newtype MaybeT m a = MaybeT { runMaybeT :: m (Maybe a) }

In [3]:
import Control.Applicative (liftA2)

instance Functor m => Functor (MaybeT m) where
    fmap f (MaybeT ma) = MaybeT $ (fmap . fmap) f ma
    
instance Applicative m => Applicative (MaybeT m) where
    pure = MaybeT . pure . pure
    
    (MaybeT mm'a2b) <*> (MaybeT mm'a) = MaybeT $ liftA2 (<*>) mm'a2b mm'a

In [4]:
instance Monad m => Monad (MaybeT m) where
    (MaybeT mma) >>= a2mmmb = MaybeT $ mma >>= ma2mmb where
--         ma2mmb = fmap join . traverse (runMaybeT . a2mmmb)
        ma2mmb Nothing = return Nothing
        ma2mmb (Just a) = (runMaybeT . a2mmmb) a

In [5]:
import Control.Monad (join)

newtype EitherT e m a = EitherT { runEitherT :: m (Either e a) }

instance Functor m => Functor (EitherT e m) where
    fmap a2b (EitherT mea) = EitherT $ (fmap . fmap) a2b mea
    
instance Applicative m => Applicative (EitherT e m) where
    pure = EitherT . pure . pure
    
    (EitherT me'a2b) <*> (EitherT me'a) = EitherT $ liftA2 (<*>) me'a2b me'a
    
instance Monad m => Monad (EitherT e m) where
    (EitherT mea) >>= a2emeb = EitherT $ mea >>= ea2meb where
--         ea2meb = fmap join . traverse (runEitherT . a2emeb)
        ea2meb (Left e) = return $ Left e
        ea2meb (Right a) = (runEitherT . a2emeb) a

---

In [6]:
newtype Compose f g a = Compose { getCompose :: f (g a) } deriving (Eq, Show)

instance (Functor f, Functor g) => Functor (Compose f g) where
    fmap f (Compose fga) = Compose $ (fmap . fmap) f fga
    
instance (Applicative f, Applicative g) => Applicative (Compose f g) where
    pure = Compose . pure . pure
    (Compose f) <*> (Compose a) = Compose $ liftA2 (<*>) f a
    
instance (Monad f, Monad g, Traversable g) => Monad (Compose f g) where
    (Compose fga) >>= a2cfgb = Compose $ fga >>= ga2fgb where
        ga2fgb = fmap join . traverse (getCompose . a2cfgb)
   
-- this instance is fine too, but we can't define two instances
-- instance (Monad f, Monad g, Cotraversable f) => Monad (Compose f g) where
--     (Compose fga) >>= a2cfgb = Compose $ fga >>= ga2fgb where
--         ga2fgb = fmap join . collect (getCompose . a2cfgb)

In [7]:
type EitherT' e m a = Compose m (Either e) a

eitherT' :: m (Either e a) -> EitherT' e m a
eitherT' = Compose

runEitherT' :: EitherT' e m a -> m (Either e a)
runEitherT' = getCompose

fmap (++"hello") (eitherT' [Left "qwer", Right "ggg"])
(eitherT' [Left "qwer", Right "ggg"]) >>= (eitherT' . const [Right "hi", Left "another"])

Line 10: Redundant bracket
Found:
(eitherT' [Left "qwer", Right "ggg"]) >>=
  (eitherT' . const [Right "hi", Left "another"])
Why not:
eitherT' [Left "qwer", Right "ggg"] >>=
  (eitherT' . const [Right "hi", Left "another"])

Compose {getCompose = [Left "qwer",Right "ggghello"]}

Compose {getCompose = [Left "qwer",Right "hi",Left "another"]}

In [8]:
swapEither :: Either e a -> Either a e
swapEither (Left e) = Right e
swapEither (Right a) = Left a

In [9]:
swapEitherT :: Functor m => EitherT e m a -> EitherT a m e
swapEitherT (EitherT mea) = EitherT $ swapEither <$> mea

In [10]:
a2mc :: a -> m c
a2mc = undefined

b2mc :: b -> m c
b2mc = undefined

:t either a2mc b2mc

either a2mc b2mc :: forall a b (m :: * -> *) c. Either a b -> m c

In [11]:
eitherT :: Monad m => (a -> m c) -> (b -> m c) -> EitherT a m b -> m c
eitherT a2mc b2mc (EitherT meab) = meab >>= either a2mc b2mc

Here *EitherT* acts exactly as *Compose* because first *Either* is applied and **then** *m*. But not all monad transformers are like so:

In [12]:
-- here it is `Reader (m a)` and NOT `m (Reader a)` so we can't use our `Compose`
newtype ReaderT r m a = ReaderT { runReaderT :: r -> m a }

In [13]:
instance Functor m => Functor (ReaderT r m) where
    fmap a2b (ReaderT rma) = ReaderT $ (fmap . fmap) a2b rma
    
instance Applicative m => Applicative (ReaderT r m) where
    pure = ReaderT . pure . pure
    
    (ReaderT rm'a2b) <*> (ReaderT rm'a) = ReaderT $ liftA2 (<*>) rm'a2b rm'a

In [14]:
instance Monad m => Monad (ReaderT r m) where
    (ReaderT rma) >>= a2rrmb = ReaderT $ \r -> rma r >>= (($ r) . runReaderT . a2rrmb)

In [15]:
-- Reader has a "superpower" that no matter how many nested Readers it has, we can "pop" it
-- to the top-level Reader
superRjoin :: Functor m => (r -> m (r -> a)) -> r -> m a
superRjoin r2m'r2a r = ($ r) <$> r2m'r2a r

sequenceR :: Functor m => m (r -> a) -> r -> m a
sequenceR m'r2a r = ($ r) <$> m'r2a

collectReader :: Functor m => (a -> r -> b) -> m a -> r -> m b
collectReader ar2b ma r = (`ar2b` r) <$> ma

instance Monad m => Monad (ReaderT r m) where
    (ReaderT rma) >>= a2rrmb = ReaderT $ rma >>= ma2rmb where
        ma2rmb = fmap join . collectReader (runReaderT . a2rrmb)

In [16]:
import Data.Cotraversable

instance Monad m => Monad (ReaderT r m) where
    (ReaderT rma) >>= a2rrmb = ReaderT $ rma >>= ma2rmb where
        ma2rmb = fmap join . collect (runReaderT . a2rrmb)

In [17]:
-- same
:t \f -> sequenceA . fmap f
:t traverse

-- same
:t \f -> cosequence . fmap f
:t collect

\f -> sequenceA . fmap f :: forall (t :: * -> *) (f :: * -> *) a1 a2. (Traversable t, Applicative f) => (a1 -> f a2) -> t a1 -> f (t a2)

traverse :: forall (t :: * -> *) (f :: * -> *) a b. (Traversable t, Applicative f) => (a -> f b) -> t a -> f (t b)

\f -> cosequence . fmap f :: forall (f :: * -> *) (g :: * -> *) a1 a2. (Cotraversable f, Functor g) => (a1 -> f a2) -> g a1 -> f (g a2)

collect :: forall (f :: * -> *) (g :: * -> *) a b. (Cotraversable f, Functor g) => (a -> f b) -> g a -> f (g b)

---

In [18]:
newtype StateT s m a = StateT { runStateT :: s -> m (a, s) }

In [19]:
import Data.Bifunctor (first)
import Data.Biapplicative (bipure)

instance Functor m => Functor (StateT s m) where
    fmap a2b (StateT s2mas) = StateT $ fmap (first a2b) . s2mas
    
instance Applicative m => Applicative (StateT s m) where
    pure x = StateT $ pure . bipure x
    --(StateT s2m'a2b's) <*> (StateT s2m'a's) = StateT $ 
    
--instance Monad m => Monad (StateT s m) where
    